<a href="https://colab.research.google.com/github/kienprojects/Machine-Learning-course/blob/main/Regression_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đề bài về nhà

## Yêu cầu

- Tự viết code cho mô hình Linear Regression theo công thức đã được dạy trong buổi lý thuyết trên lớp.
- Tự viết hàm dự đoán.
- Huấn luyện cả mô hình của thư viện và mô hình mình tự viết.
- In ra các trọng số: w0, w1, w2, ..., wn của cả 2 mô hình đã huấn luyện để quan sát và so sánh.
- Dự đoán dữ liệu tập test bằng cả 2 mô hình (mô hình thư viện thì dùng hàm predict() của thư viện, mô hình tự viết dùng hàm dự đoán tự viết), in ra kết quả bằng Dataframe như trong bài thực hành trên lớp.
- Tính RMSE trên tập test cho cả 2 mô hình và so sánh.

## Dữ liệu

In [1]:
#Cài đặt phiên bản scikit-learn cũ hơn để lấy dữ liệu về giá nhà ở Boston
!pip install scikit-learn==1.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.15.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.1.3 which is incompatible.


Tập dữ liệu giá nhà ở Boston đã có sẵn trên sklearn, dữ liệu đã được chuẩn hóa và chia thành tập train, tập test

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Đọc dữ liệu

Dữ liệu về giá nhà ở Boston được hỗ trợ bởi sklearn, đọc dữ liệu thông qua hàm `datasets.load_boston()`

Xem thêm các bộ dữ liệu khác tại https://scikit-learn.org/stable/datasets/index.html#toy-datasets.

Dữ liệu được chia thành các thành phần data và target như tập diabetes. Dữ liệu cũng đã được chuẩn hóa, chỉ cần gọi ra và huấn luyện

In [3]:
# lay du lieu dataset - du lieu ve giá nhà
dataset = datasets.load_boston()
print("Số chiều dữ liệu input: ", dataset.data.shape)
print("Số chiều dữ liệu target: ", dataset.target.shape)
print()

print("5 mẫu dữ liệu đầu tiên:")
print("input: ", dataset.data[:5])
print("target: ",dataset.target[:5])

Số chiều dữ liệu input:  (506, 13)
Số chiều dữ liệu target:  (506,)

5 mẫu dữ liệu đầu tiên:
input:  [[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 7.1850e+00
  6.1100e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9283e+02
  4.0300e+00]
 [3.2370e-02 0.0000e+00 2.1800e+00 0.0000e+00 4.5800e-01 6.9980e+00
  4.5800e+01 6.0622e+00 3.0000e+00 2.2200e+02 1.8700e+01 3.9463e+02
  2.9400e+00]
 [6.9050e-02 0.0000e+00 2.1800e+00 0.0000e+00 4.5800e-01 7.1470e+00
  5.4200e+01 6.0622e+00 3.0000e+00 2.2200e+02 1.8700e+01 3.9690e+02
  5.3300e+00]]
target:  [24.  21.6 34.7 33.4 36.2]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

**Chia dữ liệu làm 2 phần training 362 mẫu và testing 80 mẫu**

In [27]:
# cat nho du lieu, lay 1 phan cho qua trinh thu nghiem,
# chia train test cac mau du lieu
# dataset_X = dataset.data[:, np.newaxis, 2]
dataset_X = dataset.data

dataset_X_train = dataset_X[:404]
dataset_y_train = dataset.target[:404]

dataset_X_test = dataset_X[405:]
dataset_y_test = dataset.target[405:]

In [38]:
# Chuẩn hóa dữ liệu
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dataset_X_train = scaler.fit_transform(dataset_X_train)
dataset_X_test = scaler.transform(dataset_X_test)

# Xây dựng mô hình

## Xây dựng mô hình bằng thư viện

In [39]:
#build linear regression using scikit-learn
model_lib = linear_model.LinearRegression()
model_lib.fit(dataset_X_train, dataset_y_train)
print("Scikit-learn Intercept (w0):", model_lib.intercept_)
print("Scikit-learn Coefficients (w1, w2, ...):", model_lib.coef_)

LinearRegression()

## Xây dựng mô hình Linear Regression tự viết

In [40]:
#build Linear Regression from scratch
class LinearRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            y_predicted = np.dot(X, self.weights) + self.bias

            dw = (1/n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1/n_samples) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


## Hàm test mô hình tự viết

In [41]:
#test Linear Regression from scratch
def test_model(X_test, y_test, weights, bias):
    y_pred = np.dot(X_test, weights) + bias
    mse = np.mean((y_test - y_pred)**2)
    rmse = np.sqrt(mse)
    return rmse

# Huấn luyện mô hình

## Huấn luyện mô hình của thư viện

In [42]:
#train Linear Regression
model_lib = linear_model.LinearRegression()
model_lib.fit(dataset_X_train, dataset_y_train)



LinearRegression()

## Training mô hình bằng Linear regression tự viết

In [68]:
#train scratch LR
model_custom = LinearRegression(learning_rate=0.05, n_iterations=1000)
model_custom.fit(dataset_X_train, dataset_y_train)

# Dự đoán các mẫu dữ liệu

## Dự đoán các mẫu dữ liệu theo mô hình của thư viện

In [69]:
y_pred_lib = model_lib.predict(dataset_X_test)

## Dự đoán các mẫu dữ liệu tính theo linear regression tự viết

In [70]:
y_pred_custom = model_custom.predict(dataset_X_test)

## Đánh giá mô hình linear regression của thư viện

In [71]:
rmse_lib = np.sqrt(mean_squared_error(dataset_y_test, y_pred_lib))
print("\nRMSE của mô hình thư viện:", rmse_lib)


RMSE của mô hình thư viện: 5.749521870254043


## Đánh giá mô hình linear regression tự viết

In [72]:
rmse_custom = test_model(dataset_X_test, dataset_y_test, model_custom.weights, model_custom.bias)
print("RMSE của mô hình tự viết:", rmse_custom)

# So sánh RMSE
print("\nSo sánh RMSE:")
print(f"Mô hình thư viện: {rmse_lib}")
print(f"Mô hình tự viết: {rmse_custom}")
print(f"Chênh lệch: {abs(rmse_lib - rmse_custom)}")

RMSE của mô hình tự viết: 5.74080261545375

So sánh RMSE:
Mô hình thư viện: 5.749521870254043
Mô hình tự viết: 5.74080261545375
Chênh lệch: 0.008719254800293008
